https://www.wrl.unsw.edu.au/news/coastsat-how-to-estimate-beach-slopes-using-satellite-imagery
https://www.wrl.unsw.edu.au/research/coastsat


In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
import time
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
from datetime import datetime
from coastsat import SDS_earthcache_api, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects, SDS_earthcache_client

## 1. Retrieval of the images from Earthcache

****this part is what is different --> using earthcache to retrieve images****

In [8]:
#setup monthly pipelines
def fmt(x):
    if x < 10:
        return '0' + str(x)
    else:
        return str(x)
def monthly_pipes(startyear,endyear=2024):  
    starts = [] ; ends = []
    dates = range(startyear, endyear)  
    for y in dates:
        for m in range(1, 13):
            starts.append(str(y) + '-' + fmt(m) + '-01')
            if m == 2:
                ends.append(str(y) + '-' + fmt(m) + '-28')
            elif m in [4, 6, 9, 11]:
                ends.append(str(y) + '-' + fmt(m) + '-30')
            else:
                ends.append(str(y) + '-' + fmt(m) + '-31')
    return starts, ends

In [7]:
# these coordinates are a rectangle around the island of Madhirivaadhoo
aoi = { "type": "Polygon", 
        "coordinates": [
      [
        [
          72.94835170895487,
          5.464405141242935
        ],
        [
          72.94835170895487,
          5.452180140477154
        ],
        [
          72.96171014465997,
          5.452180140477154
        ],
        [
          72.96171014465997,
          5.464405141242935
        ],
        [
          72.94835170895487,
          5.464405141242935
        ]
      ]
    ]
    }

['2023-01-31',
 '2023-02-28',
 '2023-03-31',
 '2023-04-30',
 '2023-05-31',
 '2023-06-30',
 '2023-07-31',
 '2023-08-31',
 '2023-09-30',
 '2023-10-31',
 '2023-11-30',
 '2023-12-31']

In [3]:
# region of interest (longitude, latitude)
# change this depending on where we want images of
# can also use SDS_tools to read from geojson or kml
key = input("please enter your API key for Earthcache:")
resolution = [ 'low' ]
location_name = 'Madhirivaadhoo_'

# need to specify what type of images want to be downloaded
image_type = "a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b" #ID for true color image
# the available IDs can be found at: https://support.skywatch.com/hc/en-us/articles/7297565063067-Available-Outputs
# look at the SDS_earthcache_api.py file for more options
# like searching, creating a pipeline from a search, and getting the max cost of a pipeline

In [4]:
# need to initialize the api first
SDS_earthcache_api.initialize(key)
#create a set of dates so each pipeline is one month
starts,ends = monthly_pipes(2023)

#get all imagery for each month
for i in range(len(starts)):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date
    SDS_earthcache_api.retrieve_images_earthcache(key, pipeline_name, aoi, start_date, end_date, image_type)
    #add 10 seconds of delay between each request
    time.sleep(10)

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
201 {'data': {'id': '3b8fe191-ead6-11ee-bb2d-539166b49226', 'name': 'Madhirivaadhoo_2023-01-01', 'start_date': '2023-01-01', 'end_date': '2023-01-30', 'aoi': {'type': 'Polygon', 'coordinates': [[[72.94835170895487, 5.464405141242935], [72.94835170895487, 5.452180140477154], [72.96171014465997, 5.452180140477154], [72.96171014465997, 5.464405141242935], [72.94835170895487, 5.464405141242935]]]}, 'area_km2': 2.0145276052499086, 'cloud_cover_percentage': 100, 'min_aoi_coverage_percentage': 0, 'interval': '30d', 'resolution_low': 30, 'resolution_high': 10, 'status': 'pending', 'tags': None, 'created_at': '2024-03-25T18:33:53.775090Z', 'updated_at': '2024-03-25T18:33:53.775090Z', 'max_cost': 0, 'result_delivery': {'max_latency': '0d', 'priorities': ['lowest_cost']}, 'output': {'id': 'a8fc3dde-a3e8-11e7-9793-ae4260ee3b4b', 'format': 'geotiff', 'mosaic': 'stitched'}}}
200 {'data': {'id': '3b8fe191-ead6-11ee-bb2d-539166b49226', 'org_id': '001JR00000AfBY1YAN

In [ ]:
# after about 20 minutes, make sure the status is ready
SDS_earthcache_api.initialize(key)
for i in range(len(starts)):
    start_date = starts[i]
    end_date = ends[i]
    pipeline_name = location_name + start_date
    status,res = SDS_earthcache_api.checkStatus(pipeline_name)
    print(status,res)

In [25]:
# once the status is good, run this cell to download the images
SDS_earthcache_api.initialize(key)
images = []
for i in range(len(starts)):
    start_date = starts[i]
    pipeline_name = location_name + start_date
    status,res = SDS_earthcache_api.checkStatus(pipeline_name)
    if(status == 200):
        images.append(SDS_earthcache_api.download_images(pipeline_name))

d5431cc4-b43b-4f31-8db4-d1efe9e589c9
data/Madhirivaadhoo_2023-01-01\108196dc-030b-4591-954e-4af1b0227500
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/3b8fe191-ead6-11ee-bb2d-539166b49226/94de387a-03d9-4939-ac35-846b002e1041/metadata/SKYWATCH_S2_MS_20230111T0521_TC_Tile_0_0_a852_metadata.json
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/3b8fe191-ead6-11ee-bb2d-539166b49226/94de387a-03d9-4939-ac35-846b002e1041/analytic/SKYWATCH_S2_MS_20230111T0521_TC_Tile_0_0_a852.tif
data/Madhirivaadhoo_2023-03-01\41393180-9f01-418c-8a3a-9ed703c0d8e5
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/4d49a314-ead6-11ee-95e5-539166b49226/8dbd32d0-ae53-4828-9288-96742c8533e9/metadata/SKYWATCH_S2_MS_20230327T0516_TC_Tile_0_0_0dfc_metadata.json
... downloading https://s3-us-west-2.amazonaws.com/aoi-processed-images-prod/4d49a314-ead6-11ee-95e5-539166b49226/8dbd32d0-ae53-4828-9288-96742c8533e9/analytic/SKYWATCH_S2_MS_20

In [ ]:
# this function visualizes the first image
SDS_earthcache_api.view_first_image(images)